In [2]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

In [1]:
def load_data_for_split(file_path):
    texts, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            text = line.replace(' ', '')
            label = []
            i = 0
            while (i < len(text)):
                if text[i] in ['，', '。', '？', '！']:
                    label.append('1')
                else:
                    label.append('0')
                i += 1
            text_list = list(text)
            # label[-1] = 2
            # print(text, label)
            # break
            texts.append(text)
            labels.append(label)

            if (len(text_list) != len(label)):
                print('Error:', text, label)
    return texts, labels

In [3]:
texts, labels = load_data_for_split('data/train_large_2.txt')

In [4]:
print(texts[:5])
print(labels[:5])

['忽忽幾晨昏，離別間之，疾病間之，不及終年同靜好。', '煢煢小兒女，孱羸若此，嬌憨若此，更煩二老費精神。', '毋人負我，毋我負人，柳下雖和有介稱，先生字此，可以諡此。', '愛老臣少，愛少臣老，馮唐爲郎無倦意，吾輩慕之，不能效之。', '深院落滕花，石不點頭龍不語。']
[['O', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '。'], ['O', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '。'], ['O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '。'], ['O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', '。'], ['O', 'O', 'O', 'O', 'O', '，', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '。']]


In [5]:
print(type(labels))
# print(np.array(labels).shape)
if isinstance(labels, list):
    labels = [str(label) for label in labels]  # Convert all elements to strings if they are not already

<class 'list'>


In [6]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [7]:
groups = []
for i in range(len(encoded_labels)):
    groups.append(i)

In [8]:
sgkf = StratifiedGroupKFold(n_splits=5)

In [9]:
def data_generator(texts, labels, batch_size):
    for start in range(0, len(texts), batch_size):
        end = min(start + batch_size, len(texts))
        yield texts[start:end], labels[start:end]

In [10]:
cnt_train = 0
cnt_valid = 0

batch_size = 10000  # Adjust batch size based on your memory capacity

for i, (train_index, val_index) in enumerate(sgkf.split(texts, encoded_labels, groups)):
    print("Length train index: ", len(train_index))
    print("Length val index: ", len(val_index))
    
    train_gen = data_generator([texts[i] for i in train_index], [encoded_labels[i] for i in train_index], batch_size)
    val_gen = data_generator([texts[i] for i in val_index], [encoded_labels[i] for i in val_index], batch_size)
    
    with open(f'data/split2/train_{i}.txt', 'w', encoding='utf-8') as train_file:
        for X_train_batch, y_train_batch in train_gen:
            for text, label in zip(X_train_batch, y_train_batch):
                train_file.write(''.join(text) + '\n')
                cnt_train += 1

    with open(f'data/split2/valid_{i}.txt', 'w', encoding='utf-8') as val_file:
        for X_val_batch, y_val_batch in val_gen:
            for text, label in zip(X_val_batch, y_val_batch):
                val_file.write(''.join(text) + '\n')
                cnt_valid += 1

print(cnt_train, cnt_valid)

c:\Users\Nhat Hung\AppData\Local\Programs\Orange\lib\site-packages\sklearn\model_selection\_split.py:885: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


MemoryError: Unable to allocate 20.7 GiB for an array with shape (592088, 4698) and data type float64